# SQL

Nous utiliserons le dialecte Transact-SQL (T-SQL) : [https://docs.microsoft.com/fr-fr/sql/t-sql/language-reference?view=sql-server-ver15](https://docs.microsoft.com/fr-fr/sql/t-sql/language-reference?view=sql-server-ver15)  

## Création BDD

In [1]:
IF (EXISTS (SELECT [name] FROM [master].[sys].[databases] WHERE [name] = 'ImdbLike'))
    DROP DATABASE [ImdbLike]

CREATE DATABASE [ImdbLike]

GO

USE [ImdbLike]

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:01

## Schéma

Un SGBD =\> plusieurs BDD

Une BDD =\> Plusieurs schémas

```
SELECT [...] FROM [data].[Table]
```

Ici, `[data]` est le schéma de `[Table]`

Les schémas servent à regrouper des tables de même nature =\> on peut y affecter des droits (ex: `[ref]` (tables de référence) en RW pour l'admin mais lecture seule pour app, `[data]` (tables de données) en RW pour admin et app mais lecture seule pour le service compta)

In [2]:
CREATE SCHEMA [ref]

GO

CREATE SCHEMA [data]

GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.010

## Création de table

In [3]:
CREATE TABLE [ref].[Genre_GRE] -- [ref] car lecture-seule pour app
(
    [Id] INT IDENTITY(1, 1) PRIMARY KEY,
    [Nom] VARCHAR(25) NOT NULL
)

CREATE TABLE [data].[Film_FLM] -- [data] car lecture-écriture pour app
(
    [Id] BIGINT IDENTITY(1, 1),-- PRIMARY KEY, -- [Id] entier 64bits auto-incrément à partir de 1 par pas de 1, clé primaire
    [Nom] NVARCHAR(80) NOT NULL, -- NVARCHAR gère l'encode, VARCHAR non -> prend 2x plus de place
    [Descriptif] NVARCHAR(MAX) NOT NULL, -- Equivalent à TEXT -> remplacer TEXT par NVARCHAR
    -- [Genre] VARCHAR(20) NOT NULL -> Non, on normalise
    [GRE_Id] INT NOT NULL-- FOREIGN KEY REFERENCES [ref].[Genre_GRE]([Id])
    CONSTRAINT PK_FLM PRIMARY KEY ([Id]) -- On privilégie cette syntaxe -> Contrainte nommée + clé composite
    CONSTRAINT FK_FLM_GRE FOREIGN KEY ([GRE_Id]) REFERENCES [ref].[Genre_GRE]([Id])
)

CREATE TABLE [data].[Acteur_ACT] (
    [Id] UNIQUEIDENTIFIER NOT NULL DEFAULT NEWID() PRIMARY KEY, -- Id 128bits sous forme de chaîne
    [Nom] NVARCHAR(80) NOT NULL,
    [Prenom] NVARCHAR(80) NOT NULL
)

CREATE TABLE [data].[FilmActeur_FAC] (
    [FLM_Id] BIGINT NOT NULL,
    [ACT_Id] UNIQUEIDENTIFIER NOT NULL
    CONSTRAINT PK_FilmActeur PRIMARY KEY ([FLM_Id], [ACT_Id]),
    CONSTRAINT FK_FilmActeur_Acteur FOREIGN KEY ([ACT_Id]) REFERENCES [data].[Acteur_ACT]([Id]),
    CONSTRAINT FK_FilmActeur_Film FOREIGN KEY ([FLM_Id]) REFERENCES [data].[Film_FLM]([Id])
)


Commands completed successfully.

Total execution time: 00:00:00.037

In [4]:
SELECT NEWID()

(1 row affected)

Total execution time: 00:00:00.719

(No column name)
8d4a2685-c584-40a3-9dad-6fc9504d98b9


## Modification de table

### Exercice

Modéliser des sociétés de production (ex: Lucas Film), les associées aux films (vous aurez besoin de l'instruction [`ALTER TABLE`](https://docs.microsoft.com/fr-fr/sql/t-sql/statements/alter-table-transact-sql?view=sql-server-ver15)). On partira du principe que un film possède une seule société de prod.

In [5]:
CREATE TABLE [data].[SocieteProd_SPR] (
    [Id] BIGINT IDENTITY(1, 1) PRIMARY KEY,
    [Nom] NVARCHAR(100) NOT NULL
)

GO

ALTER TABLE [data].[Film_FLM]
    ADD [SPR_Id] BIGINT NOT NULL
    CONSTRAINT FK_FML_SPR FOREIGN KEY ([SPR_Id]) REFERENCES [data].[SocieteProd_SPR]([Id])

GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.016

## Opérations CRUD

Create, Read, Update, Delete

  

Create -\> INSERT

Read -\> SELECT

Update -\> UPDATE

Delete -\> DELETE

In [6]:
INSERT INTO [ref].[Genre_GRE] ([Nom])
VALUES ('Science Fiction'), ('Action'), ('Aventure')

INSERT INTO [data].[SocieteProd_SPR] ([Nom])
VALUES (N'Lucas Film'), (N'Gaumont')

INSERT INTO [data].[Acteur_ACT] ([Nom], [Prenom])
VALUES (N'Ford', N'Harrisson'), (N'Depp', N'Johny')

(3 rows affected)

(2 rows affected)

(2 rows affected)

Total execution time: 00:00:00.020

In [7]:
INSERT INTO [data].[Film_FLM] ([Nom], [Descriptif], [GRE_Id], [SPR_Id])
VALUES
    (N'Indiana Jones', N'Descriptif du film', 3, 1),
    (N'Pirates de Caraïbes', N'Descriptif de l''autre film', 2, 2)

(2 rows affected)

Total execution time: 00:00:00.022

In [8]:
INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 1
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Ford'

INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 2
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Depp'

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.032

In [9]:
UPDATE [data].[Film_FLM]
SET [Descriptif] = N'Film sur un archéologue'
WHERE [Id] = 1

UPDATE [data].[Film_FLM]
SET [Descriptif] = N'Film de pirates'
WHERE [Id] = 2


(1 row affected)

(1 row affected)

Total execution time: 00:00:00.010

In [10]:
DELETE FROM [ref].[Genre_GRE]
WHERE [Id] = 1

(1 row affected)

Total execution time: 00:00:00.014

## Vues

Reppel jointures : https://sql.sh/cours/jointures

In [11]:
SELECT [FLM].[Id], [FLM].[Nom], [GRE].[Nom] AS [Genre], [SPR].[Nom] AS [Société de prod]
FROM [data].[Film_FLM] AS [FLM]
INNER JOIN [ref].[Genre_GRE] AS [GRE] ON [FLM].[GRE_Id] = [GRE].[Id]
INNER JOIN [data].[SocieteProd_SPR] AS [SPR] ON [FLM].[SPR_Id] = [SPR].[Id]

(2 rows affected)

Total execution time: 00:00:00.021

Id,Nom,Genre,Société de prod
1,Indiana Jones,Aventure,Lucas Film
2,Pirates de Caraïbes,Action,Gaumont


In [12]:
CREATE OR ALTER VIEW [data].[FicheFilm]
AS
SELECT [FLM].[Id], [FLM].[Nom], [GRE].[Nom] AS [Genre], [SPR].[Nom] AS [Société de prod]
FROM [data].[Film_FLM] AS [FLM]
INNER JOIN [ref].[Genre_GRE] AS [GRE] ON [FLM].[GRE_Id] = [GRE].[Id]
INNER JOIN [data].[SocieteProd_SPR] AS [SPR] ON [FLM].[SPR_Id] = [SPR].[Id]

Commands completed successfully.

Total execution time: 00:00:00.009

In [13]:
SELECT [Id], [Nom], [Genre], [Société de prod]
FROM [data].[FicheFilm]

(2 rows affected)

Total execution time: 00:00:00.015

Id,Nom,Genre,Société de prod
1,Indiana Jones,Aventure,Lucas Film
2,Pirates de Caraïbes,Action,Gaumont


### Exercice

1\. Insérer de nouveaux acteurs (4), genres, sociétés de prod (3) et de nouveaux films (5) dans vos tables (assignez des acteurs aux films).

2\. Créer une vue qui sort les acteurs avec leurs films associés (Nom de l'acteur, nom du film, genre du film, société de prod)

In [14]:
INSERT INTO [ref].[Genre_GRE] ([Nom])
VALUES ('Horreur'), ('Science Fiction'), ('Comédie')

INSERT INTO [data].[SocieteProd_SPR] ([Nom])
VALUES (N'Disney'), (N'Netflix'), (N'Pathé')

INSERT INTO [data].[Acteur_ACT] ([Nom], [Prenom])
VALUES (N'De Niro', N'Robert'), (N'Fois', N'Marina'), (N'Cotillard', N'Marion'), (N'Nanty', N'Isabelle')

INSERT INTO [data].[Film_FLM] ([Nom], [Descriptif], [GRE_Id], [SPR_Id])
VALUES
    (N'Les tuches', N'Description tuches', 6, 5),
    (N'Batman', N'Descriptif Batman', 5, 4),
    (N'Rrrrr', N'Descriptif Rrrrr', 6, 1),
    (N'Casino', N'Descriptif Casino', 2, 1),
    (N'Pirates de Caraïbes 2', N'Descriptif de l''autre film', 2, 3)

(3 rows affected)

(3 rows affected)

(4 rows affected)

(5 rows affected)

Total execution time: 00:00:00.028

In [15]:
INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 3
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Nanty'

INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 4
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Cotillard'

INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 5
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Fois'

INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 6
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'De Niro'

INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 7
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Depp'

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.037

In [16]:
CREATE OR ALTER VIEW [data].[ViewActeursFilms] WITH SCHEMABINDING
AS
    SELECT [ACT].[Nom], [FLM].[Nom] AS [Film], [GRE].[Nom] AS [Genre], [SPR].[Nom] AS [Prod]
    FROM [data].[Acteur_ACT] AS [ACT]
    INNER JOIN [data].[FilmActeur_FAC] AS [FAC] ON [FAC].[ACT_Id] = [ACT].[Id]
    INNER JOIN [data].[Film_FLM] AS [FLM] ON [FLM].[Id] = [FAC].[FLM_Id]
    INNER JOIN [ref].[Genre_GRE] AS [GRE] ON [GRE].[Id] = [FLM].[GRE_Id]
    INNER JOIN [data].[SocieteProd_SPR] AS [SPR] ON [SPR].[Id] = [FLM].[SPR_Id]

Commands completed successfully.

Total execution time: 00:00:00.009

In [17]:
SELECT [Nom], [Film], [Genre], [Prod] FROM [data].[ViewActeursFilms]
ORDER BY [Nom]

(7 rows affected)

Total execution time: 00:00:00.035

Nom,Film,Genre,Prod
Cotillard,Batman,Science Fiction,Netflix
De Niro,Casino,Action,Lucas Film
Depp,Pirates de Caraïbes 2,Action,Disney
Depp,Pirates de Caraïbes,Action,Gaumont
Fois,Rrrrr,Comédie,Lucas Film
Ford,Indiana Jones,Aventure,Lucas Film
Nanty,Les tuches,Comédie,Pathé


## Vues Matérialisées

Une vue à un instant T écrite sur disque. =\> Pas à jour en temps réel.

Point fort : Vitesse de lecture instantanée

Point faible : Mise à jour manuelle

## Index

Créer sur des tables ou des vues.

Différents types d'index :

\- Gestion de contraintes (unicité d'une valeur)

\- Faciliter l'accès aux données

\- ...

In [18]:
CREATE UNIQUE INDEX [IDX_Unique_GRE] ON [ref].[Genre_GRE]([Nom] DESC)

Commands completed successfully.

Total execution time: 00:00:00.015

In [19]:
--INSERT INTO [ref].[Genre_GRE] ([Nom])
--VALUES ('Horreur') -- Index unique, on a déjà la valeur -> erreur

Commands completed successfully.

Total execution time: 00:00:00.003

In [20]:
CREATE INDEX [IDX_Acteur_Id_Par_Nom] ON [data].[Acteur_ACT]([Nom], [Id])
CREATE UNIQUE CLUSTERED INDEX [IDX_Vue_Acteur_Film] ON [data].[ViewActeursFilms]([Nom], [Film])

Commands completed successfully.

Total execution time: 00:00:00.026

In [21]:
SELECT [Id]
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Depp'

(1 row affected)

Total execution time: 00:00:00.019

Id
80e6e67c-d150-4b16-ad4d-7f9e69b1bc42


In [22]:
SELECT [Film]
FROM [data].[ViewActeursFilms]
WHERE [Nom] = 'Depp'

(2 rows affected)

Total execution time: 00:00:00.014

Film
Pirates de Caraïbes
Pirates de Caraïbes 2


## Fonctions

2 types :

\- Natives

\- UDF (User Defined Functions)

In [23]:
SELECT TOP 1
    GETDATE() AS [Date],
    DATEADD(DAY, 2, GETDATE()) AS [Après demain],
    DATEDIFF(DAY, DATEADD(MONTH, 2, GETDATE()), GETDATE()) AS [Diff deux mois],
    CONCAT([Prenom], ' ', [Nom]) AS [Nom complet],
    COALESCE(null, 'Test') AS [Coalesce]
FROM [data].[Acteur_ACT]

(1 row affected)

Total execution time: 00:00:00.018

Date,Après demain,Diff deux mois,Nom complet,Coalesce
2021-02-17 18:26:07.787,2021-02-19 18:26:07.787,-59,Robert De Niro,Test


In [24]:
CREATE FUNCTION [data].[fun_GetFilmParNomActeur](@Nom NVARCHAR(80))
RETURNS NVARCHAR(80)
AS
BEGIN
    DECLARE @Film NVARCHAR(80)

    SELECT TOP 1  @Film = [Film]
    FROM [data].[ViewActeursFilms]
    WHERE [Nom] = @Nom

    RETURN @Film
END

Commands completed successfully.

Total execution time: 00:00:00.007

In [25]:
SELECT [Nom], [data].[fun_GetFilmParNomActeur]([Nom]) AS [Film]
FROM [data].[Acteur_ACT]

(6 rows affected)

Total execution time: 00:00:00.015

Nom,Film
Cotillard,Batman
De Niro,Casino
Depp,Pirates de Caraïbes
Fois,Rrrrr
Ford,Indiana Jones
Nanty,Les tuches


## Exercice

\- Créez une table de référence récompenses avec les valeurs "Oscar du meilleur acteur", "Oscar de la meilleure actrice", "Meilleure bande son"

\- Faire en sorte qu'un film ou un acteur pluisse avoir 0 à n récompenses

\- Attribuer des récompenses à deux acteurs et deux films (si possible pas à un acteur du film)

\- Faire une vue indéxée qui permet de récupérer les récompenses des acteurs/films

| Nom            | Récompense               |
|----------------|--------------------------|
| Robert De Niro | Oscar du meilleur acteur |
| Les Tuches     | Meilleure bande son      |
| Batman         | NULL                     |